# How to Manage Customers

> Create and manage customer records in Stripe

This guide shows you how to create and manage customer records using FastStripe.

## Problem

You need to store customer information, track their payment history, and manage their subscriptions.

## Solution

Use FastStripe's customer management methods to create, update, and retrieve customer data.

In [ ]:
from faststripe.core import StripeApi
import os

sapi = StripeApi(os.environ['STRIPE_SECRET_KEY'])

## Creating customers

In [ ]:
# Create a basic customer
customer = sapi.customers.create(
    email='john.doe@example.com',
    name='John Doe'
)

print(f"Created customer: {customer.id}")
print(f"Customer email: {customer.email}")

## Creating customers with full details

In [ ]:
# Create customer with address and metadata
detailed_customer = sapi.customers.create(
    email='jane.smith@example.com',
    name='Jane Smith',
    phone='+1234567890',
    address={
        'line1': '123 Main St',
        'city': 'San Francisco',
        'state': 'CA',
        'postal_code': '94105',
        'country': 'US'
    },
    metadata={
        'user_id': '12345',
        'plan': 'premium',
        'signup_date': '2024-01-15'
    }
)

print(f"Customer with details: {detailed_customer.id}")
print(f"Address: {detailed_customer.address.line1}, {detailed_customer.address.city}")
print(f"Metadata: {detailed_customer.metadata}")

## Finding customers

In [ ]:
# Find customer by email
customers = sapi.customers.fetch(email='john.doe@example.com')

if customers.data:
    found_customer = customers.data[0]
    print(f"Found customer: {found_customer.name} ({found_customer.email})")
else:
    print("Customer not found")

## Using helper methods

In [ ]:
# Use FastStripe's find_product helper
existing_customer = sapi.find_product('John Doe')  # This will need to be adapted for customers

# List recent customers
recent_customers = sapi.customers.fetch(limit=5)
print(f"Recent customers: {len(recent_customers.data)}")

for customer in recent_customers.data:
    print(f"- {customer.name or 'No name'} ({customer.email})")

## Updating customers

In [ ]:
# Update customer information
updated_customer = sapi.customers.update(
    customer.id,
    description='Premium plan customer',
    metadata={
        'plan': 'premium',
        'updated_date': '2024-01-20'
    }
)

print(f"Updated customer description: {updated_customer.description}")

## Customer payment methods

In [ ]:
# List customer's payment methods
payment_methods = sapi.payment_methods.fetch(
    customer=customer.id,
    type='card'
)

print(f"Customer has {len(payment_methods.data)} payment methods")

for pm in payment_methods.data:
    if pm.card:
        print(f"- {pm.card.brand} ending in {pm.card.last4}")

## Customer subscriptions

In [ ]:
# Get customer's subscriptions
subscriptions = sapi.subscriptions.fetch(customer=customer.id)

print(f"Customer has {len(subscriptions.data)} subscriptions")

for sub in subscriptions.data:
    print(f"- Subscription {sub.id}: {sub.status}")
    if sub.items.data:
        item = sub.items.data[0]
        amount = item.price.unit_amount / 100
        print(f"  Amount: ${amount:.2f}/{item.price.recurring.interval}")

## Customer invoices

In [ ]:
# Get customer's invoices
invoices = sapi.invoices.fetch(customer=customer.id, limit=5)

print(f"Customer has {len(invoices.data)} recent invoices")

for invoice in invoices.data:
    amount = invoice.amount_paid / 100
    print(f"- Invoice {invoice.number}: ${amount:.2f} ({invoice.status})")

## Best practices

- Always store your internal user ID in the customer's metadata
- Use meaningful descriptions for easier identification in the dashboard
- Keep customer email addresses up to date for receipts and communication
- Use the address field for tax calculation and compliance
- Consider customer data privacy and GDPR compliance when storing information